In [73]:
import config
import mysql.connector
import requests
import time
import yelp_config
import json

In [74]:
client_id = yelp_config.my_id
api_key = yelp_config.api

In [ ]:
### business params and url
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)

    data = response.json()['businesses']
    return data


def all_results(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 50:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        time.sleep(1)  # Wait a second
        cur += 50
    print('all_results worked')
    return results


term = 'burgers'
location = 'Brooklyn NY'
url_params = {'term': term.replace(' ', '+'),
              'location': location.replace(' ', '+'),
              'limit': 50
              }

#raw_business_data = all_results(url_params, api_key)

In [105]:
#extracting business and skimmed the data the I need
def datum_dict(business_dict):
    ''' this function receives a dictionary with all the yelp keys and returns
        a new dictionary with only the name, id, rating and price. and a dictionary with only the business keys'''
    data_dict = {}
    for k, v in business_dict.items():
        if k == 'id':
            data_dict[k] = v
        elif k == 'name':
            data_dict[k] = v
        elif k == 'rating':
            data_dict[k] = v
        elif k == 'price':
            data_dict[k] = v
    return data_dict

def get_dict_from_data(dict_list):
    '''this function returns a a dictionary from a list of dictionary'''
    business_list = []
    for a_list in dict_list:
        for business in a_list:
            try:
                business_list.append(datum_dict(business))
            except:
                cbusiness_listontinue
    return business_list

# get list of all the businesses dictionary's
skim_business_data = get_dict_from_data(raw_business_data)
two = skim_business_data[:2]
print(two)

[{'id': 'dkAj-3gmkvdA4XkJmw6hCw', 'name': "Juanchi's Burger", 'rating': 4.5, 'price': '$$'}, {'id': 'Jy6qxXks0w5Wmurzu7rmBQ', 'name': '212 Burgers', 'rating': 4.5, 'price': '$$'}]


In [107]:
def get_reviews(biz_id):
    url = "https://api.yelp.com/v3/businesses/{}/reviews".format(biz_id)
    headers = {'Authorization': 'Bearer {}'.format(yelp_config.api)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['reviews']
    return data

def get_all_reviews(b_ids):
    reviews = []
    for b_id in b_ids:
        try:
            reviews.append(get_one_review(b_id))
        except:
            print(f'failed to get the review for {b_id}')
    return reviews

def create_review_structure(business_dict):
    ids_list = []
    for biz in business_dict:
        ids_list.append(biz['id'])
    
    reviews_list = []
    
    for an_id in ids_list:
        skimmed_reviews = {}
        full_reviews = get_reviews(an_id)
        for every_review in full_reviews:
            print(every_review)
            skimmed_reviews['review_id'] = every_review['id']
            skimmed_reviews['time_created'] = every_review['time_created']
            skimmed_reviews['business_id'] = an_id
            reviews_list.append(skimmed_reviews)
            
        
    return reviews_list
skim_review_data = create_review_structure(two)

{'id': 'JRCYCUMpYWZnR_e-_AIK-A', 'url': 'https://www.yelp.com/biz/juanchis-burger-brooklyn?adjust_creative=KcsTlt_o2HYJz1wDnNimtg&hrid=JRCYCUMpYWZnR_e-_AIK-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KcsTlt_o2HYJz1wDnNimtg', 'text': "Made a Yelp reservation for Saturday at 6:45pm. They had a table blocked off for us with a 'reserved' sign. That was nice! Place is kind of small, maybe...", 'rating': 5, 'time_created': '2019-07-22 08:39:35', 'user': {'id': 'eoxP7LZmxINRAmXWhsq8ww', 'profile_url': 'https://www.yelp.com/user_details?userid=eoxP7LZmxINRAmXWhsq8ww', 'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/dFuyYgUInpaC1D_9DlB-SA/o.jpg', 'name': 'Meghan R.'}}
{'id': 'bnsrQSIc_2OJQdbVkCtTJg', 'url': 'https://www.yelp.com/biz/juanchis-burger-brooklyn?adjust_creative=KcsTlt_o2HYJz1wDnNimtg&hrid=bnsrQSIc_2OJQdbVkCtTJg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KcsTlt_o2HYJz1wDnNimtg', 'text': 'We were craving burgers and luckily

In [94]:
print(skim_review_data)
# create business query, add the data and commit query
# connecting my db and start inserting data into it
cnx = mysql.connector.connect(
    host=config.host,
    user=config.user,
    passwd=config.password,
    database=config.DB_NAME
)
cursor = cnx.cursor()
add_review = ("INSERT INTO business "
              "(review_id, time_created, business_id) "
              "VALUES (%s, %s, %s)")

def convert_to_tuple(skim_review_data):
    try:
        datum = (skim_review_data['review_id'], skim_review_data['time_created'],
                 skim_review_data['business_id'])
        

    except:
        print(f'unable to create tuple for {skim_review_data}')
    return datum
              
def create_data(business_data):
    data = []
    for biz_dict in business_data:
        data.append(convert_to_tuple(biz_dict))
    return data

converted_data = create_data(skim_review_data)
              
              
def db_insertion(query, data):
    for datum in data:
        try:
            cursor.execute(query, datum)
            cnx.commit()
            print('great success!')
        except:
            pass
    return
              
db_insertion(add_review, converted_data)              
# Make sure data is committed to the database
cnx.commit()
# make sure the connection is closed.
cursor.close()
cnx.close()

[{'review_id': 'hLSb6MUv3-P-3Cd0ZptajA', 'time_created': '2019-05-25 06:44:00', 'business_id': '-0bdnX762vdTb9lI00etvA'}, {'review_id': 'hLSb6MUv3-P-3Cd0ZptajA', 'time_created': '2019-05-25 06:44:00', 'business_id': '-0bdnX762vdTb9lI00etvA'}, {'review_id': 'hLSb6MUv3-P-3Cd0ZptajA', 'time_created': '2019-05-25 06:44:00', 'business_id': '-0bdnX762vdTb9lI00etvA'}, {'review_id': 'hLSb6MUv3-P-3Cd0ZptajA', 'time_created': '2019-05-25 06:44:00', 'business_id': '-0bdnX762vdTb9lI00etvA'}, {'review_id': 'hLSb6MUv3-P-3Cd0ZptajA', 'time_created': '2019-05-25 06:44:00', 'business_id': '-0bdnX762vdTb9lI00etvA'}, {'review_id': 'hLSb6MUv3-P-3Cd0ZptajA', 'time_created': '2019-05-25 06:44:00', 'business_id': '-0bdnX762vdTb9lI00etvA'}, {'review_id': 'hLSb6MUv3-P-3Cd0ZptajA', 'time_created': '2019-05-25 06:44:00', 'business_id': '-0bdnX762vdTb9lI00etvA'}, {'review_id': 'hLSb6MUv3-P-3Cd0ZptajA', 'time_created': '2019-05-25 06:44:00', 'business_id': '-0bdnX762vdTb9lI00etvA'}, {'review_id': 'hLSb6MUv3-P-3Cd0